In [1]:
import cv2
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
# import tensorflow as tf
from tensorflow import keras as K
import skimage
from skimage.io import imshow, imread, imsave
from glob import glob
import os

2024-09-16 20:52:38.436079: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# import pkg_resources

# for dist in pkg_resources.working_set:
#     if 'numpy' in dist.requires():
#         print(dist.project_name)

In [3]:
in_dir = '../dataset/Original'
out_dir = './clustered'

# read the input image file names with paths into a list
infiles = in_dir + '/*'
img_names = glob(infiles)
# print(img_names)

In [4]:
K=3
attempts=10

for item in img_names:
    print('processing %s...' % item)

    img = cv2.imread(item)
    img2 = img.reshape((-1,3))
    img2=np.float32(img2)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    ret,label,center=cv2.kmeans(img2,K,None,criteria,attempts,cv2.KMEANS_PP_CENTERS)
    center=np.uint8(center)
    res = center[label.flatten()]
    res2 = res.reshape((img.shape))


    # write the result to disk in the previously created output directory
    name = os.path.basename(item)
    outfile = out_dir + '/' + name
    cv2.imwrite(outfile, res2)

processing ../dataset/Original/fbcb2ef0-3c97-4d7a-8b96-0147c73f3941.png...
processing ../dataset/Original/fc15659f-6264-4dd5-ab17-43d89fb297b9.png...
processing ../dataset/Original/fc8df086-39e0-43f8-b0c7-2d23a349539b.png...
processing ../dataset/Original/fcc2b245-f47f-447f-8527-275b0005677c.png...
processing ../dataset/Original/fcdf6632-c03f-49a8-9fb3-42e87c99b9d0.png...
processing ../dataset/Original/fd1302e2-3cd2-4a1a-9c13-00f810a9c41b.png...
processing ../dataset/Original/fd1c4c7a-24bc-43f2-b84f-fee182207aee.png...
processing ../dataset/Original/fde0fd54-49cc-42e1-90c6-6b612cd062d8.png...
processing ../dataset/Original/fe00fa09-d3c8-485b-9d0f-3cce556c9edd.png...
processing ../dataset/Original/fe0c5e48-f65e-488c-9c3c-1ea7c7835c0c.png...
processing ../dataset/Original/fe1ee954-ba71-47b3-954a-d20ab940cd7b.png...
processing ../dataset/Original/fe436c09-b680-4f4c-8faf-aa10a55a335b.png...
processing ../dataset/Original/fe691faa-3bc7-475e-b558-aa8fe7b795bc.png...
processing ../dataset/Ori

In [5]:
in_dir = './clustered'
out_dir = './predicted_mask'

# read the input image file names with paths into a list
infiles = in_dir + '/*'
img_names = glob(infiles)
# print(img_names)

In [6]:
for item in img_names:
    print('processing %s...' % item)

    # read an input image as gray
    im_gray = cv2.imread(item,0)
    
    #blur = cv2.GaussianBlur(im_gray,(5,5),0)

    # threshold it with your saved threshold
    try:
        im_bw = cv2.threshold(im_gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    except Exception as e:
        print(f"An error occurred: {e}")

    # write the result to disk in the previously created output directory
    name = os.path.basename(item)
    outfile = out_dir + '/' + name
    cv2.imwrite(outfile, im_bw)

processing ./clustered/fe0c5e48-f65e-488c-9c3c-1ea7c7835c0c.png...
processing ./clustered/fe1ee954-ba71-47b3-954a-d20ab940cd7b.png...
processing ./clustered/fe436c09-b680-4f4c-8faf-aa10a55a335b.png...
processing ./clustered/fe691faa-3bc7-475e-b558-aa8fe7b795bc.png...
processing ./clustered/fe851c88-692d-4199-87e0-d19d9c4eb591.png...
processing ./clustered/fed6ba05-36a5-45dc-a4e1-9baa7de2c622.png...
processing ./clustered/ff1b23d0-3cef-459b-b81f-39bfcdb6526f.png...
processing ./clustered/ff37dd9b-2e3c-444d-9d29-6db186cfa2fb.png...
processing ./clustered/ff3b9d4d-373c-48ef-a75a-d23e2c2ade87.png...
processing ./clustered/ffb7be9a-65df-4f0b-99ae-cfd34e81cd34.png...
processing ./clustered/ffd59802-46c6-4b58-80fe-e534e39781a7.png...
processing ./clustered/f8b6a868-60e9-4569-ae69-638a0c39834d.png...
processing ./clustered/f8ebe0c8-a90d-4a64-9f6a-f9985e38c340.jpg...
processing ./clustered/f91670db-be74-474c-bd72-68120aed4756.png...
processing ./clustered/f939782b-f5a2-40a3-9c22-6c6d7e18edc3.pn

In [7]:
def calculate_dice_coefficient(mask1, mask2, file1, file2):
    # Flatten the masks to 1D arrays
    mask1 = mask1.flatten()
    mask2 = mask2.flatten()
    
    # Calculate intersection and sizes of the masks
    if(mask1.shape != mask2.shape):
        print(f"Shape mismatch for {file1}, {file2}: original_mask shape {mask1.shape}, kmeans_mask shape {mask2.shape}")
    intersection = np.sum((mask1 == 255) & (mask2 == 255))  # White pixels in both masks
    dice_coeff = (2.0 * intersection) / (np.sum(mask1 == 255) + np.sum(mask2 == 255))
    
    return dice_coeff, intersection, np.sum(mask1 == 255) + np.sum(mask2 == 255)

def calculate_jaccard_index(mask1, mask2):
    # Flatten the masks to 1D arrays
    mask1 = mask1.flatten()
    mask2 = mask2.flatten()
    
    # Calculate intersection and union
    intersection = np.sum((mask1 == 255) & (mask2 == 255))  # White pixels in both masks
    union = np.sum((mask1 == 255) | (mask2 == 255))         # White pixels in either mask
    
    jaccard_index = intersection / union
    
    return jaccard_index, intersection, union

In [8]:
# Paths to the folders containing the masks
original_mask_dir = '../dataset/Mask'
kmeans_mask_dir = 'predicted_mask'


original_mask_files = sorted(glob(os.path.join(original_mask_dir, '*')))
kmeans_mask_files = sorted(glob(os.path.join(kmeans_mask_dir, '*')))

dice_scores = []
jaccard_scores = []

for original_mask_file, kmeans_mask_file in zip(original_mask_files, kmeans_mask_files):
    original_mask = cv2.imread(original_mask_file, 0)  # Read as grayscale
    kmeans_mask = cv2.imread(kmeans_mask_file, 0)      # Read as grayscale
    
    # # Ensure the masks are binary (0 or 255)
    # _, original_mask = cv2.threshold(original_mask, 127, 255, cv2.THRESH_BINARY)
    # _, kmeans_mask = cv2.threshold(kmeans_mask, 127, 255, cv2.THRESH_BINARY)
    
    # Calculate Dice Coefficient and Jaccard Index for the current image
    dice, intersection_dice, union_dice = calculate_dice_coefficient(original_mask, kmeans_mask, original_mask_file, kmeans_mask_file)
    jaccard, intersection_jaccard, union_jaccard = calculate_jaccard_index(original_mask, kmeans_mask)
    
    # Append scores for averaging approach
    dice_scores.append(dice)
    jaccard_scores.append(jaccard)
    
    # Accumulate values for combined approach
    # total_intersection_dice += intersection_dice
    # total_union_dice += union_dice
    # total_intersection_jaccard += intersection_jaccard
    # total_union_jaccard += union_jaccard

# Average Dice and Jaccard
avg_dice = np.mean(dice_scores)
avg_jaccard = np.mean(jaccard_scores)

# Combined Dice and Jaccard
# combined_dice = (2.0 * total_intersection_dice) / total_union_dice
# combined_jaccard = total_intersection_jaccard / total_union_jaccard

print(f'Average Dice Coefficient for all images: {avg_dice:.4f}')
print(f'Average Jaccard Index for all images: {avg_jaccard:.4f}')

# print(f'Combined Dice Coefficient for all images: {combined_dice:.4f}')
# print(f'Combined Jaccard Index for all images: {combined_jaccard:.4f}')

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known inc

Average Dice Coefficient for all images: 0.9338
Average Jaccard Index for all images: 0.8792
